In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes accelerate==0.20.3  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install datasets==2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-rluwy3go
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-rluwy3go
  Resolved https://github.com/huggingface/transformers.git to commit 18ee1fe76295239335bf1528c744fe1cfba21cc8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7639845 sha256=f4b797417e0d056a96b5074806460860a77aa9a898212c2918d7343e15c055bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-6zx746br/wheels/cf/59/82/6492402e887a68975030bf8c06532260abc16abb7ccd8127cc
Successfully built transformers
  Attempting uninstall: tra

  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-45axu3g_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-45axu3g_
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggingface/peft.git e536616888d51b453ed354a6f1e243fecb02ea08
  Running command git checkout -q e536616888d51b453ed354a6f1e243fecb02ea08
  Resolved https://github.com/huggingface/peft.git to commit e536616888d51b453ed354a6f1e243fecb02ea08
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=41637 sha256=31d0612ebc992af1de2b879b55b347ba0044620cdfad694eb2c6f75e37c4bf93
  Stored in directory: /root/.cache/pip/wheels/02/0a/9a/b9755f6b184f58a5a44

In [62]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

In [74]:
!ls -a
!du -sh .ipynb_checkpoints
!du -sh .config
!ls -a ..
!du -sh ../content
!du -sh ../opt
!du -sh ../home

.	 .ipynb_checkpoints		 predicted-llama7b-sql.txt
..	 predicted-llama7b-sql_old2.txt  sample_data
.config  predicted-llama7b-sql_old.txt	 validation_llama_format.json
4.0K	.ipynb_checkpoints
120K	.config
.			    .dockerenv	mnt			  srv
..			    etc		NGC-DL-CONTAINER-LICENSE  sys
bin			    home	opt			  tmp
boot			    lib		proc			  tools
content			    lib32	python-apt		  usr
cuda-keyring_1.0-1_all.deb  lib64	root			  var
datalab			    libx32	run
dev			    media	sbin
56M	../content
984M	../opt
4.0K	../home


In [66]:
from huggingface_hub import notebook_login
notebook_login()

In [67]:
base_model = "meta-llama/Llama-2-13b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")
tokenizer.pad_token = tokenizer.eos_token

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:563 in          │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   560 │   │   │   )                                                                              │
│   561 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   562 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 563 │   │   │   return model_class.from_pretrained(                                            │
│   564 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   565 │   │   │   )                                                                              │
│   566 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3114 in from_pretrained   │
│                                                                                                  │
│   3111 │   │   │   │   │   key: device_map[key] for key in device_map.keys() if key not in modu  │
│   3112 │   │   │   │   }                                                                         │
│   3113 │   │   │   │   if "cpu" in device_map_without_lm_head.values() or "disk" in device_map_  │
│ ❱ 3114 │   │   │   │   │   raise ValueError(                                                     │
│   3115 │   │   │   │   │   │   """                                                               │
│   3116 │   │   │   │   │   │   Some modules are dispatched on the CPU or the disk. Make sure yo  │
│   3117 │   │   │   │   │   │   the quantized model. If you want to dispatch the model on the CP  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to 
fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a 
custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-
cpu-and-gpu
                        for more details.

In [64]:
import json

# Open the JSON file
with open('/content/validation_llama_format.json') as f:
  val_spider_data = json.load(f)

# val_spider_data = val_spider_data[:150]
len(val_spider_data)

1034

In [ ]:
def generate_responses(prompts, model, tokenizer, max_new_tokens=100, device="cuda"):
    # Convert list of prompts to batch of model inputs
    model_inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)

    # Move the inputs to the specified device
    model_inputs = {key: val.to(device) for key, val in model_inputs.items()}

    # Set the model to evaluation mode
    model.eval()

    # Initialize an empty list to store the responses
    responses = []

    # Perform inference without computing gradients
    with torch.no_grad():
        # Generate output
        outputs = model.generate(**model_inputs, max_new_tokens=max_new_tokens, early_stopping=True, pad_token_id=tokenizer.eos_token_id)

        # Loop through each output in the batch
        for i, output in enumerate(outputs):
            # Get the length of the corresponding input
            input_length = model_inputs["input_ids"].shape[1]

            # Decode only the generated part of the output and add it to the responses list
            responses.append(tokenizer.decode(output[input_length:], skip_special_tokens=True))

    return responses

In [ ]:
def format_prompt(data_point):
  return f"""{data_point["input"]}

### Response:
"""

In [ ]:
formatted_list = list(map(format_prompt, val_spider_data))
formatted_list[1]

'You are a powerful text-to-SQL model. Here is a database schema:\nstadium :\nStadium_ID [ INT ] primary_key\nLocation [ TEXT ]\nName [ TEXT ]\nCapacity [ INT ]\nHighest [ INT ]\nLowest [ INT ]\nAverage [ INT ]\n\nsinger :\nSinger_ID [ INT ] primary_key\nName [ TEXT ]\nCountry [ TEXT ]\nSong_Name [ TEXT ]\nSong_release_year [ TEXT ]\nAge [ INT ]\nIs_male [ bool ]\n\nconcert :\nconcert_ID [ INT ] primary_key\nconcert_Name [ TEXT ]\nTheme [ TEXT ]\nStadium_ID [ TEXT ] concert.Stadium_ID = stadium.Stadium_ID\nYear [ TEXT ]\n\nsinger_in_concert :\nconcert_ID [ INT ] primary_key singer_in_concert.concert_ID = concert.concert_ID\nSinger_ID [ TEXT ] singer_in_concert.Singer_ID = singer.Singer_ID\n\nWrite an SQL query that answers the following. Please respond with only SQL: What is the total number of singers? \n\n### Response:\n'

In [ ]:
def batch_list(lst, batch_size):
  """Split lst into batches of size batch_size"""

  batched_list = []

  for i in range(0, len(lst), batch_size):
    batch = lst[i:i+batch_size]
    batched_list.append(batch)

  return batched_list

In [ ]:
batched = batch_list(formatted_list, 4)

In [ ]:
def get_first_line(s):
    """Returns the first line of the input string."""
    return s.split('\n')[0]

In [59]:
def append_to_file(lines, file_path):
    """Appends lines to a file."""
    with open(file_path, 'a') as f:
        for line in lines:
            if (line == " " or line == "" or line == "\n"):
              line = "EMPTY RESPONSE"
            f.write(line + '\n')

In [ ]:
batch = batched[0]
print(batch)
responses = generate_responses(batch, model, tokenizer, max_new_tokens=50)
# for response in responses:
#   print("repsonse is: ", response)
# # print(responses)
# first_lines = list(map(get_first_line, responses))
# print("First lines are: ", first_lines)
# # Append first_lines to a text file
# append_to_file(first_lines, 'predicted-llama7b-sql.txt')

['You are a powerful text-to-SQL model. Here is a database schema:\nstadium :\nStadium_ID [ INT ] primary_key\nLocation [ TEXT ]\nName [ TEXT ]\nCapacity [ INT ]\nHighest [ INT ]\nLowest [ INT ]\nAverage [ INT ]\n\nsinger :\nSinger_ID [ INT ] primary_key\nName [ TEXT ]\nCountry [ TEXT ]\nSong_Name [ TEXT ]\nSong_release_year [ TEXT ]\nAge [ INT ]\nIs_male [ bool ]\n\nconcert :\nconcert_ID [ INT ] primary_key\nconcert_Name [ TEXT ]\nTheme [ TEXT ]\nStadium_ID [ TEXT ] concert.Stadium_ID = stadium.Stadium_ID\nYear [ TEXT ]\n\nsinger_in_concert :\nconcert_ID [ INT ] primary_key singer_in_concert.concert_ID = concert.concert_ID\nSinger_ID [ TEXT ] singer_in_concert.Singer_ID = singer.Singer_ID\n\nWrite an SQL query that answers the following. Please respond with only SQL: How many singers do we have? \n\n### Response:\n', 'You are a powerful text-to-SQL model. Here is a database schema:\nstadium :\nStadium_ID [ INT ] primary_key\nLocation [ TEXT ]\nName [ TEXT ]\nCapacity [ INT ]\nHighest 

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
print(batch[0])
print("LHuillier response is:")
print(responses[0])

You are a powerful text-to-SQL model. Here is a database schema:
stadium :
Stadium_ID [ INT ] primary_key
Location [ TEXT ]
Name [ TEXT ]
Capacity [ INT ]
Highest [ INT ]
Lowest [ INT ]
Average [ INT ]

singer :
Singer_ID [ INT ] primary_key
Name [ TEXT ]
Country [ TEXT ]
Song_Name [ TEXT ]
Song_release_year [ TEXT ]
Age [ INT ]
Is_male [ bool ]

concert :
concert_ID [ INT ] primary_key
concert_Name [ TEXT ]
Theme [ TEXT ]
Stadium_ID [ TEXT ] concert.Stadium_ID = stadium.Stadium_ID
Year [ TEXT ]

singer_in_concert :
concert_ID [ INT ] primary_key singer_in_concert.concert_ID = concert.concert_ID
Singer_ID [ TEXT ] singer_in_concert.Singer_ID = singer.Singer_ID

Write an SQL query that answers the following. Please respond with only SQL: How many singers do we have? 

### Response:

LHuillier response is:

```
SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert
```

### Explanation:

We have 10 singers.

### Solution:

```
SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert
```


In [60]:
import re

def extract_code_block(s):
    """Extracts the content inside the first set of triple backticks in the input string."""
    match = re.search(r'```(.*?)```', s, re.DOTALL)
    output = match.group(1) if match else get_first_line(s)
    print("Output is: ", output)
    output = output.replace("\n", " ")
    return output

In [61]:
for batch in batched:
    print(batch)
    responses = generate_responses(batch, model, tokenizer, max_new_tokens=50)
    for response in responses:
      print("repsonse is: ", response)
    # print(responses)
    first_lines = list(map(extract_code_block, responses))
    print("First lines are: ", first_lines)
    # Append first_lines to a text file
    append_to_file(first_lines, '/content/predicted-llama13b-sql.txt')
    # break

['You are a powerful text-to-SQL model. Here is a database schema:\nstadium :\nStadium_ID [ INT ] primary_key\nLocation [ TEXT ]\nName [ TEXT ]\nCapacity [ INT ]\nHighest [ INT ]\nLowest [ INT ]\nAverage [ INT ]\n\nsinger :\nSinger_ID [ INT ] primary_key\nName [ TEXT ]\nCountry [ TEXT ]\nSong_Name [ TEXT ]\nSong_release_year [ TEXT ]\nAge [ INT ]\nIs_male [ bool ]\n\nconcert :\nconcert_ID [ INT ] primary_key\nconcert_Name [ TEXT ]\nTheme [ TEXT ]\nStadium_ID [ TEXT ] concert.Stadium_ID = stadium.Stadium_ID\nYear [ TEXT ]\n\nsinger_in_concert :\nconcert_ID [ INT ] primary_key singer_in_concert.concert_ID = concert.concert_ID\nSinger_ID [ TEXT ] singer_in_concert.Singer_ID = singer.Singer_ID\n\nWrite an SQL query that answers the following. Please respond with only SQL: How many singers do we have? \n\n### Response:\n', 'You are a powerful text-to-SQL model. Here is a database schema:\nstadium :\nStadium_ID [ INT ] primary_key\nLocation [ TEXT ]\nName [ TEXT ]\nCapacity [ INT ]\nHighest 

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


repsonse is:  
```
SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert
```

### Explanation:

We have 10 singers.

### Solution:

```
repsonse is:  
```
SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert
```

### Explanation:

The query is counting the number of distinct singers in the concert table.


repsonse is:  
| Name | Country | Age |
| --- | --- | --- |
| Adele | UK | 27 |
| Celine Dion | Canada | 46 |
| Shakira | Colombia | 
repsonse is:  
```
SELECT
    s.Name,
    s.Country,
    s.Age
FROM
    singer s
INNER JOIN
    singer_in_concert sic ON s.Singer_ID = sic.
Output is:  
SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert

Output is:  
SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert

Output is:  
Output is:  
First lines are:  [' SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert ', ' SELECT COUNT(DISTINCT Singer_ID) FROM singer_in_concert ', '', '']
['You are a powerful text-to-SQL model. Here is a database schema:\nstadium :\nStadium_ID [ INT ] 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:3                                                                              │
│ in generate_responses:17                                                                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1602 in generate        │
│                                                                                                  │
│   1599 │   │   │   )                                                                             │
│   1600 │   │   if generation_mode == GenerationMode.GREEDY_SEARCH:                               │
│   1601 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1602 │   │   │   return self.greedy_search(                                                    │
│   1603 │   │   │   │   input_ids,                                                                │
│   1604 │   │   │   │   logits_processor=logits_processor,                                        │
│   1605 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2450 in greedy_search   │
│                                                                                                  │
│   2447 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2448 │   │   │                                                                                 │
│   2449 │   │   │   # forward pass to get next token                                              │
│ ❱ 2450 │   │   │   outputs = self(                                                               │
│   2451 │   │   │   │   **model_inputs,                                                           │
│   2452 │   │   │   │   return_dict=True,                                                         │
│   2453 │   │   │   │   output_attentions=output_attentions,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

In [ ]:
# eval_prompt1 = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

# You must output the SQL query that answers the question.
# ### Input:
# Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

# ### Context:
# CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

# ### Response:
# """

# eval_prompt2 = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

# You must output the SQL query that answers the question.
# ### Input:
# Which Class has a Frequency MHz larger than 91.5, and a City of license of birmingham?

# ### Context:
# CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

# ### Response:
# """

# eval_prompt3 = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

# You must output the SQL query that answers the question.
# ### Input:
# Which Class has a Frequency MHz larger than 91.5, and a City of license of new york?

# ### Context:
# CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

# ### Response:
# """

# eval_prompt4 = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

# You must output the SQL query that answers the question.
# ### Input:
# Which Class has a Frequency MHz larger than 91.5, and a City of license of austin?

# ### Context:
# CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

# ### Response:
# """

# # List of input prompts
# prompts = [
#     eval_prompt1,
#     eval_prompt2,
#     eval_prompt3,
#     eval_prompt4
#     # ... (add more prompts as needed)
# ]

# # Generate responses for the list of prompts
# responses = generate_responses(prompts, model, tokenizer)
# for response in responses:
#     print("\nNew Response:")
#     print(response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



New Response:
SELECT * FROM table_name_12 WHERE class > '91.5' AND city_of_license = 'hyannis'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context

New Response:
SELECT * FROM table_name_12 WHERE class > '91.5' AND city_of_license = 'birmingham'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of birmingham?

### Context:
CREATE TABLE table_

New Response:
SELECT * FROM table_name_12 WHERE class > '91.5' AND city_of_license = 'new york'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of new york?

### Context:
CREATE TABLE

New Response:
SELECT * FROM table_name_12 WHERE class = 'class' AND frequency_mhz > '91.5' AND city_of_license = 'austin'

### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of austin?


In [ ]:
# print(len(responses))
# response[3]
# # for response in responses:
# #     print("New Response:")
# #     print(response)

4


'E'